# Talktorial 12

# Binding Site Detection (DogSiteScorer)

Abishek Laxmanan Ravi Shankar

## Aim of the talktorial

To get a protein structure and **detect its ligand binding site** using **DoGSiteScorer** at the example of **EGFR** and validate the detected binding site with **KLIFS** and **ligand based detection**. 

Be more precise (can be done once you outlined the practical).

# Learning goals

### Theory
* Protein binding sites
* Binding site detection 
    * Introduction
    * DogSiteScorer    
* Validation
    * Ligand-based detection
    * KLIFS pocket definitions

### Practical

TBA

## References

* Combining global and local measures for structure-based druggability predictions.
([<i>Noordwijkerhout Cheminformatics</i> 2011, <b>52</b>, 360-372](https://www.ncbi.nlm.nih.gov/pubmed/22148551))

* Prediction, Analysis, and Comparison of Active Sites 
([ <i>Journal of Chemical Information and Modeling</i> 2018 <b>6,7</b>](https://onlinelibrary.wiley.com/doi/abs/10.1002/9783527806539.ch6g))

* DoGSiteScorer
([<i>Bioinformatics</i> 2012, <b>28</b>, 2074-2075](https://doi.org/10.1093/bioinformatics/bts310)) 

* Fpocket
([<i>BMC Bioinformatics</i> 2009, <b>10</b>, 168](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-10-168))

* Protein–ligand complex
([Protein complex wiki](https://en.wikipedia.org/wiki/Protein%E2%80%93ligand_complex))

* Voronoi tessellation
([Voronoi tessellation Wiki](https://en.wikipedia.org/wiki/Voronoi_diagram))

* KLIFS database
( [KLIFS Website](http://klifs.vu-compmedchem.nl/))  

* Fragments based drug Discovery
([Google books](https://books.google.de/books?id=tjfvCQAAQBAJ&pg=PA202&lpg=PA202&dq=Fragments+based+drug+Discovery+SURFNET&source=bl&ots=O6XVGMVPTV&sig=ACfU3U2K6btUeMpgN4FBe627jpIJL6-mtQ&hl=en&sa=X&ved=2ahUKEwjonezYmbLkAhXBRxUIHdzKClIQ6AEwAnoECAkQAQ))

* KLIFS: a structural kinase-ligand interaction database
([<i>KLIFS Pubmed</i> 2016, Nucleic Acids Research, <b>44</b>, 365-371](https://www.ncbi.nlm.nih.gov/pubmed/26496949)) 

* KLIFS: A Knowledge-Based Structural Database To Navigate Kinase−Ligand Interaction Space
([<i>KLIFS Journal of Medicinal Chemistry</i> 2013, <b>57</b>, 249-257](https://pubs.acs.org/doi/10.1021/jm400378w))

* SURFNET
([<i>Journal of Molecular Graphics</i>1995, <b>13</b>323-330](https://www.ncbi.nlm.nih.gov/pubmed/8603061))

* LIGSITE
([<i>Journal of Molecular Graphics and Modelling</i> 1997 <b>6</b> 359-363](https://www.sciencedirect.com/science/article/pii/S1093326398000023?via%3Dihub))

* Docking 
( [Docking wiki](https://en.wikipedia.org/wiki/Docking_(molecular)))

* DrugSite
([<i>Genome Informatics</i>2004 <b>15</b> 31–41 ](https://pdfs.semanticscholar.org/59ca/aac9a79c29660385af0ab83b82fba57cc476.pdf))

* Metapocket 2.0
([Summary of various binding site detection approaches](https://projects.biotec.tu-dresden.de/metapocket/algorithm.php))



## Theory

### Protein binding sites

The main objective of a pharmaceutical research is to find a new drug inorder to cure a specific disease. Structure-based approaches in drug design develop drugs with respect to a target protein which is associated with the disease under investigation. However, in order to develop such a drug it is crucial to know where the drug is supposed to bind in the protein and therefore inhibit or activate the protein. Not always such a small molecule binding site is known. Therefore it is important to have tools for binding site detection.

Binding site detection has a special mention in the process of drug development. Binding sites are defined as a **hollow 3 dimensional space** on the surface of a protein molecule, which serves as the region for **ligand**, **peptide**, or **protein** binding. There are various factors which influence whether such a region is suitable binding site.
* The Shape
* The Spatial arrangement of the residues
* The Possible interactions are
    * Van der Waals interactions
    * Hydrogen bond interactions
    * Hydrophobic regions
    * Electrostatic interactions
    * π-π interactions
   


### Binding site detection 

If ligand information is available (protein-ligand-complex), then the ligand-surrounding protein region is defined as pocket (e.g. using a 6 Å radius).
If the ligand is absent, then the detection tools can be used in the process of pocket detection. This comprises of 2 standard approaches. These methods can be grouped into, **geometry-based and energy-based** approaches as well as **grid-based and grid-free** approaches as outlined in the Figure 1.

<div style="width:500; font-size:100%; text-align:center;">
    <img src="images/Figure1_DetectionMethods.png" alt="alternate text"  width="500" style="padding:0.8em;"/>
    Figure 1: Binding site detection methods can be grouped into geometry-based and energy-based approaches as well as grid-based and grid-free approaches.
</div>

#### Geometry-based approaches

Geometry-based approaches analyze the shape of a molecular surface to locate cavities. It is based upon the 3D spatial arrangement of the atoms on the protein surface. They are broadly divided into two namely Grid-Based and Grid-free approaches.

* **Grid-based approaches**

    * Examples: POCKET, LIGSITE and DoGSiteScorer 
    * In LIGSITE, the cartesian coordinates runs along the X, Y and Z axes through a 3D region which consists of protein surface on both sides.
    * In addition to these axes a cubic diagonal passes along to find out the width of the cleft in between the protein surface. Therefore LIGSITE checks in seven directions the distance of each atom in the protein surface.
    * Therefore these coordinates enables LIGSITE to diffrentate the Protein Solvent Protein regions (PSP) on the protein surface
    * DoGSiteScorer is explained in detail below.
   
* **Grid-free approaches**

    * Example: SURFNET
    * In SURFNET spheres are placed on the protein surface instead of having grids. 
    * The probe spheres are placed midway between the pairs of atoms on the protein. Incase a probe overlaps any nearby atom, its radius is reduced until no overlap occurs. 
    * The resulting probes defines the pocket and the cavities. The cutoff radius of the spheres are of 1 to 4 angstrom .

#### Energy-based approaches

Interaction of a molecular fragment with protein is recorded. Favourable energitic responses are assigned to pockets. DrugSite and Docking are the well known energy-based methods. They are again divided into grid-based and grid-free approaches.
<div class="center" align="right" width="500"> </div>

* **Grid-based approaches**

    * Example: Drugsite
    * This method uses carbon probes placed on each grid point, and van der Wall's energies between the probes within a distance of 8 Å are calculated. Mean energy and Standard deviation are calculated for these grid points. 
    * Grid points with unfavourable energies are removed. Grid points fulfilling these cut-off are merged to pockets.
   
* **Grid-free approaches**

    * Example: Docking
    * Docking process is similar to "lock and key" model in which the **target protein resembles** the **lock** and **ligand** to the **key**
    * In this method, a scoring function is incorporated to evaluate the fragments that are placed at a position on the protein surface.
    * Scoring functions are physics based, which uses Molecular Mechanics force fields, which primarily estimates the amount of affinity between the binding ligand and the protein.
    * The scoring functions calculates the feasibilty of whether the ligand could be docked to the particular protein
    * These pockets are assigned based on the quantity of fragments that bind to a specific area.

### DogSiteScorer 

Certain geometric and physico-chemical properties are calculated in an automatic manner for the predicted pockets and subpockets. In **DogSiteScorer** , pockets and subpockets are predicted with DoGSite using a Difference of **Gaussian filter**. DogSiteScorer falls under the category of Geometry-based and Grid-based approaches. Pocket volume and surface are calculated by counting the **grid points** constituting the pocket volume or its surface and multiplying this number with the grid box volume or surface, respectively. A **breadth-first search** algorithm is used for pocket depth computation, starting from the solvent exposed pocket parts toward the most deeply buried regions. 

#### Pocket Detection

* The protein structure is covered around with a grid of coordinates.
* Each of the **grid points** are **labelled**.
* Difference of **Gaussian filter** is applied across the grid.
* This operation helps to find the position on a protein surface where the location of a **sphere-like object** is favorable.
* The last step in this algorithm is the merging of neighbouring subpockets to the pockets.

<div style="width:400; font-size:100%; text-align:center;">
    <img src="images/Figure2_BindingSitePocket.png" alt="alternate text"  width="400" style="padding:0.8em;"/>
    Figure 2: The binding site on the surface of the protein shown with grid view consisting of the surface molecules, volume inside the binding region, and the region exposed to the exterior of the protein 
</div>

#### Druggability 

* The Druggability is based on Machine learning technique - **Support Vector Machine (SVM)**.
* **Discriminative Analysis** - Best suited to seperate druggable from undruggable.
* The model has been trained and tested on both the redundant and non-redundant version of the druggable dataset
* This model showed a mean accuracy of 90%.
* The druggabilty score lies between 0 and 1. The higher the score the better the chance that the pocket is druggable.
    
<div style="width:1000; font-size:100%; text-align:center;">
    <img src="images/Figure3_DogSiteAlgo.png" alt="alternate text"  width="1000" style="padding:0.8em;"/>
    Figure 3: Flowsheet explaining the pipeline of DoGSiteScorer
</div>


### Validation methods

#### Ligand-based detection

Sometimes ligand information is available (protein-ligand-complex), in that case usually the ligand-surrounding protein region is defined as pocket (e.g. using a 6 Å radius).

If a bound ligand is known for a structure, it can be used for binding-site annotation.  


#### KLIFS pocket definition

KLIFS stands for **Kinase-Ligand Interaction Fingerprints and structures database**. It is a structural repository of over 2900 human and mouse kinase enzymes along with their catalytic activity. It also deals with the kinase domains and also the inhibitors which can interact with them. Kinases share a similar conserved regions, which serves as a challenge for small molecules that can selectively bind to a specific kinase. 

KLIFS enables us to do systematic comparison and analysis of chemical features of all available kinases in the process of ligand binding. The classification helps us of an all-encompassing binding site of 85 residues. It is possible to compare the **interaction patterns** of **kinase-inhibitors** to each other to, for example, identify crucial interactions determining kinase-inhibitor selectivity. Therefore this could be an ideal method used for the validation of the obtained binding site. 

## Practical

In [1]:
import time

from biopandas.mol2 import PandasMol2
from biopandas.pdb import PandasPdb
import requests
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Binding site detection using DoGSiteScorer

#### Submit job to DoGSiteScorer and get job location

In [3]:
def dogsitescorer_submit_with_pdbid(pdb_code, chain_id, ligand=''):
    """
    Submit PDB ID to DoGSiteScorer webserver using their API and get back URL for job location.
    
    Parameters
    ----------
    pdb_id : str 
        4-letter valid PDB ID, e.g. '3w32'.
    chain : str
        Chain ID, e.g. 'A'.
    ligand : str
        Name of ligand bound to PDB structure with pdb_id, e.g. 'W32_A_1101'. 
        Currently, the ligand name must be checked manually on the DoGSiteScorer website. 
        
    Returns
    -------
    str
        Job location URL for submitted query.
        
    References
    ----------
    Function is taken from: https://github.com/volkamerlab/TeachOpenCADD/pull/3
    """
    
    # Submit job to proteins.plus
    r = requests.post("https://proteins.plus/api/dogsite_rest",
        json={
            "dogsite": {
                "pdbCode": pdb_code,
                "analysisDetail": "1",
                "bindingSitePredictionGranularity": "1",
                "ligand": ligand,
                "chain": chain_id
            }
        },
        headers= {'Content-type': 'application/json', 'Accept': 'application/json'}
    )

    r.raise_for_status()
    
    return r.json()['location']

In [4]:
# Identifying the job location where the work is submitted to the web server
job_location = dogsitescorer_submit_with_pdbid('3w32', 'A')
job_location

'https://proteins.plus/api/dogsite_rest/FSAdQY1ADHYFviSbwHG2jbPK'

In [5]:
# Wait a bit so that job can finish
time.sleep(30)

#### Get DoGSiteScorer pocket metadata

In [6]:
def get_dogsitescorer_metadata(job_location):
    """
    xxx - This function retrieves results from a DoGSiteScorer query.
    It returns the binding sites which are found over the protein surface, 
    in the form of a table with the details about all detected pockets.

    Parameters
    ----------
    job_location : str
        Consists of the location of a finished DoGSiteScorer job on the proteins.plus web server.
    
    Returns
    -------
    result_table_df : pandas.DataFrame
        Table with metadata on detected binding sites. 
    """
    
    # Get job results
    result = requests.get(job_location)
    
    # Get URL of result table file
    result_file = result.json()['result_table']
    
    # Get result table
    result_table = requests.get(result_file).text
    
    # Split string into list of lists (=table)
    result_table_split = [i.split('\t') for i in result_table[:-1].split('\n')]

    # Remove spaces
    result_table_split = [[j.replace(' ', '') for j in i] for i in result_table_split]

    # Extract column names, index names, table body
    column_names = result_table_split[0]
    index_names = [i[0] for i in result_table_split[1:]]
    table = [i[1:] for i in result_table_split[1:]]

    # Convert to DataFrame
    result_table_df = pd.DataFrame(
        table,
        columns=column_names[1:],
        index=index_names
    )
    result_table_df.index.name = 'name'
    
    # Convert number strings to numeric values
    for name, data in result_table_df.iteritems():
        try:
            result_table_df[name] = pd.to_numeric(data)
        except ValueError:
            pass
    
    return result_table_df

In [7]:
# Retrieving all the guessed pockets from DoGSiteScorer Web server 
metadata = get_dogsitescorer_metadata(job_location)
metadata

KeyError: 'result_table'

#### Sort pockets based on their drugScore

In [ ]:
# Sort the obtained binding site by descending drugScore
metadata.sort_values(by='drugScore', ascending=False)

In [ ]:
def select_best_pocket(metadata, by='drugScore'):
    """
    xxx - 'drugScore' is used as a parameter to identify the best pocket among the obtained pockets.
    
    Parameters
    ----------
    metadata : pd.DataFrame
        xxx
    by : str
       xxx
        
    Returns 
    -------
    str
        The best pocket based on the 'DrugScore' is returned here as the output.
    """
    
    by_methods = ['drugScore', 'volume']
    
    # Sort by best druggability score
    if by == 'drugScore':
        sorted_pocket = metadata.sort_values(by='drugScore', ascending=False)
    # Sort by volume
    elif by == 'volume':
        sorted_pocket = metadata.sort_values(by='volume', ascending=False)
    else:
        raise ValueError(f'Selection method not in list: {", ".join(by_methods)}')
                         
    # Get name of best pocket
    best_pocket_name = sorted_pocket.iloc[0, :].name     
        
    return best_pocket_name

In [ ]:
# Get the name of the best pocket
best_pocket = select_best_pocket(metadata, by='drugScore')
best_pocket

#### Get best binding site file content

In [ ]:
def get_pocket_locations(job_location):
    """
    xxx - We have identified the best pocket by the above step. Now we must obtain the best pocket file in ".pdb" format.
    
    Parameters
    ----------
    job_location : str
        xxx
    
    Returns
    -------
    str
        xxx
    """
    
    # Get job results
    result = requests.get(job_location)
    
    # Get residues
    return result.json()['residues']

In [ ]:
def get_best_pocket_location(pocket_files, best_pocket):
    """
    xxx - To create a function to get the best binding site file from the list of all obtained pdb files.
    
    Parameters
    ----------
    pocket_files : list
        xxx
    best_pocket : str
        xxx

    Returns
    ------
    xxx
        xxx
    """
    result = []
    
    for pocket_file in pocket_files:
        
        if f'{best_pocket}_res' in pocket_file:
            result.append(pocket_file)
            
    if len(result) > 1:
        raise TypeError(f'Multiple strings detected: {", ".join(result)}.')
    elif len(result) == 0:
        raise TypeError(f'No string detected.')
    else:
        pass
            
    return result[0]

In [ ]:
# Get URL for all PDB files
pocket_files = get_pocket_locations(job_location)
pocket_files

In [ ]:
# Get URL for PDB file containing the best pocket
best_pocket_location = get_best_pocket_location(pocket_files, best_pocket)
best_pocket_location

#### Get residue IDs and names of best pocket

In [ ]:
def get_pocket_residues(pocket_location):
    """
    Get residue IDs and names of a pocket.
    
    Parameters
    ----------
    pocket_location : str
        xxx
        
    Returns
    -------
    xxx
        xxx
    """
    
    # Retrieve PDB file content from URL
    result = requests.get(pocket_location)

    # Get content of PDB file  
    pdb_residues = result.text
    
    # Load PDB format as DataFrame
    ppdb = PandasPdb()
    pdb_df = ppdb._construct_df(pdb_residues.splitlines(True))['ATOM']
    
    return pdb_df[['residue_number', 'residue_name']]

In [ ]:
# Get residues of best pocket
pocket_residues = get_pocket_residues(best_pocket_location)
pocket_residues

# Validation using KLIFS

We have identified the binding site of a kinase of interest with the help of DoGSiteScorer web server. 
Now we want to check if the obtained binding site is in agreement with the defined binding site defined by the KLIFS database. 

In [ ]:
from bravado.client import SwaggerClient

KLIFS_API_DEFINITIONS = "http://klifs.vu-compmedchem.nl/swagger/swagger.json"
KLIFS_CLIENT = SwaggerClient.from_url(KLIFS_API_DEFINITIONS, config={'validate_responses': False})

#### Get KLIFS structure ID for a PDB ID of interest 

In [ ]:
def get_klifsid_from_pdbid(pdb_codes):
    """
    Get KLIFS structure ID for a PDB ID of interest.
    
    Parameters
    ----------
    pdb_codes : str or list of str
        xxx
    
    Returns
    -------
    xxx
        xxx
    """
    
    # If only a single PDB ID is provided in the form of a string, cast string to list
    if isinstance(pdb_codes, str):
        pdb_codes = [pdb_codes]
    
    # Get metadata for KLIFS structures associated with one or more PDB IDs
    klifs_structures = KLIFS_CLIENT.Structures.get_structures_pdb_list(
        pdb_codes=pdb_codes
    ).response().result
    
    # Get KLIFS IDs for KLIFS structures
    klifs_ids = [i.structure_ID for i in klifs_structures]
    
    # Get first KLIFS ID
    klifs_id = klifs_ids[0]
    
    return int(klifs_id)

In [ ]:
# Get KLIFS ID
klifs_id = get_klifsid_from_pdbid('3w32')
klifs_id

#### Get pocket from KLIFS ID and extract residues

In [ ]:
def get_klifs_pocket(klifs_id):
    """
    xxx - This function takes structure ID aqs the input and returns the binding site residues as the output
    
    Parameters
    ----------
    klifs_id : str
        xxx
    
    Returns
    -------
    xxx
        xxx
    """
    
    klifs_pocket = KLIFS_CLIENT.Structures.get_structure_get_pocket(
        structure_ID=klifs_id
    ).response().result
    
    return klifs_pocket

In [ ]:
def get_klifs_residues(klifs_pocket):
    """
    xxx - This function returns the residue IDs and residue names which are returned from the KLIFS database 
    
    Parameters
    ----------
    klifs_pocket : str
        xxx
    
    Returns
    -------
    xxx
        xxx
    """
    
    ppdb = PandasMol2()
    mol2_df = ppdb._construct_df(
        binding_site.splitlines(True),  
        col_names=['1','2','3','4','5','6','7','Residues','9','10'],  #TODO include reasonable column names
        col_types=[str]*10  #TODO include reasonable
    )
    
    return mol2_df[['Residues']]

In [ ]:
# Get KLIFS pocket
klifs_pocket = get_klifs_pocket(klifs_id)
klifs_pocket

In [ ]:
# Get KLIFS residues
klifs_residues = get_klifs_residues(klifs_pocket)
klifs_residues

In [ ]:
# Split residue ID and name (in order to have the same output as for the DSS pocket residues)

In [ ]:
res='ALA333'

In [ ]:
res_name = res[:3]
res_name

In [ ]:
res_id = res[3:]
res_id

In [ ]:
# AL's notes
'ALA333'.replace(r'^((?:\D*\d){3})(?=.+)', r'\1.')

klifs_residues[0].str.replace(r'^((?:\D*\d){3})(?=.+)', r'\1.')

klifs_residues.Residues.apply(lambda x: pd.Series(str(x).split('7',1)))

# Split residue ID and name (in order to have the same output as for the DSS pocket residues)
kresidue_name, kresidue_id = klifs_residues[:len(klifs_residues)//2], klifs_residues[len("klifs_residues")//2:]
kresidue_name
kresidue_id

## Compare DoGSiteScorer and KLIFS pocket

Intersection of residue IDs?